In [4]:
%%writefile utils.py
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

Overwriting utils.py


In [8]:
import os
import nltk
from os import getcwd
# import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)


In [11]:
# select the set of positive and negative tweets
from nltk.corpus import twitter_samples

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [12]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [16]:
# Import the utility functions
from utils import process_tweet, build_freqs
import numpy as np

# Process tweets
all_tweets = all_positive_tweets + all_negative_tweets
ys = np.append(np.ones((len(all_positive_tweets), 1)), np.zeros((len(all_negative_tweets), 1)), axis=0)

# Note: process_tweet is called inside build_freqs, so we don't need to pre-process here.
# processed_tweets = [process_tweet(tweet) for tweet in all_tweets]

# Build frequency dictionary
freqs = build_freqs(all_tweets, ys)

print("Frequency dictionary built successfully!")
print(f"Size of frequency dictionary: {len(freqs)}")

Frequency dictionary built successfully!
Size of frequency dictionary: 13141


In [18]:
# Create label arrays for training and testing data
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

print("train_y.shape: ", train_y.shape)
print("test_y.shape: ", test_y.shape)

train_y.shape:  (8000, 1)
test_y.shape:  (2000, 1)


In [19]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11397


In [20]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [21]:
def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    ### START CODE HERE ###
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))
    ### END CODE HERE ###

    return h

In [22]:
# Testing your function
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


In [25]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Manual tests
def test_sigmoid():
    assert np.isclose(sigmoid(0), 0.5), "❌ Test failed for input 0"
    assert np.isclose(sigmoid(2), 1 / (1 + np.exp(-2))), "❌ Test failed for input 2"
    assert np.isclose(sigmoid(-2), 1 / (1 + np.exp(2))), "❌ Test failed for input -2"
    print("✅ All sigmoid tests passed!")

test_sigmoid()


✅ All sigmoid tests passed!


In [26]:
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2


np.float64(9.210340371976294)

In [27]:
# UNQ_C2 GRADED FUNCTION: gradientDescent
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):

        # get z, the dot product of x and theta
        z = np.dot(x,theta)

        # get the sigmoid of h
        h = sigmoid(z)

        # calculate the cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))

    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [28]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [np.float64(4.1e-07), np.float64(0.00035658), np.float64(7.309e-05)]


/tmp/ipython-input-27-4278095536.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [33]:
def gradientDescent(x, y, theta, alpha, num_iters, computeCost, computeGradient):
    for i in range(num_iters):
        grad = computeGradient(x, y, theta)
        theta = theta - alpha * grad
    return theta


In [34]:
# UNQ_C3 GRADED FUNCTION: extract_features
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    ### START CODE HERE ###

    # loop through each word in the list of words
    # loop through each word in the list of words
    for word in word_l:

        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)

        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)

    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [35]:
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.000e+00 3.133e+03 6.100e+01]]


In [36]:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [43]:
def extract_features(tweet, freqs):
    word_l = tweet.lower().split()
    x = np.zeros((3,))
    x[0] = 1  # bias term
    for word in word_l:
        x[1] += freqs.get((word, 1), 0)
        x[2] += freqs.get((word, 0), 0)
    return x


In [46]:
# UNQ_C2 GRADED FUNCTION: gradientDescent
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):

        # get z, the dot product of x and theta
        z = np.dot(x,theta)

        # get the sigmoid of h
        h = sigmoid(z)

        # calculate the cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))

    ### END CODE HERE ###
    J = float(J)
    return J, theta


# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.30801877.
The resulting vector of weights is [np.float64(4e-08), np.float64(0.00049776), np.float64(-0.00051491)]


/tmp/ipython-input-46-4278605255.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [47]:
# UNQ_C4 GRADED FUNCTION: predict_tweet
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###

    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)

    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))

    ### END CODE HERE ###

    return y_pred

In [48]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


I am happy -> 0.500000
I am bad -> 0.494791
this movie should have been great. -> 0.499996
great -> 0.514864
great great -> 0.529702
great great great -> 0.544487
great great great great -> 0.559195


/tmp/ipython-input-48-1003351.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


In [49]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([0.81341154])

In [54]:
# Test your function
w1_unittest=test_predict_tweet=(predict_tweet, freqs, theta)

In [55]:
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

    ### START CODE HERE ###

    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator

    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)

    ### END CODE HERE ###

    return accuracy

In [56]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9515


In [57]:
# Some error analysis done for you
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @SasaRichardson @Stefbystef_ @Frgt10_Anthem HAHAHAHAHAHAHAHAHAHAHAHAHAHA I AM DYING :)) I AM LITERALLY IN FRONT OF IT AND I WAS LIKE
THE PROCESSED TWEET IS: ['hahahahahahahahahahahahahaha', 'die', ':)', 'liter', 'front', 'like']
1	0.49944285	b'hahahahahahahahahahahahahaha die :) liter front like'
THE TWEET IS: All my Bae :)) &lt;3 
By : @Merima_Beslagic http://t.co/gDy1trnfjV
THE PROCESSED TWEET IS: ['bae', ':)', '<3']
1	0.49972112	b'bae :) <3'
THE TWEET IS: I :)"@Kreizi_: Who wants a #FF S/O??"
THE PROCESSED TWEET IS: [':)', 'want', 'ff']
1	0.49974256	b':) want ff'
THE TWEET IS: @CaballeroSerena you actually need to stop tweeting and driving! :-)))))))
THE PROCESSED TWEET IS: ['actual', 'need', 'stop', 'tweet', 'drive', ':-)']
1	0.49400619	b'actual need stop tweet drive :-)'
THE TWEET IS: @Madison420Ivy  "We" ??? you wish :D:D:D:D:D:D:D kidding
THE PROCESSED TWEET IS: ['wish', ':d', ':d', ':d', ':d', ':d', ':d', ':d', 'kid']
1	0.49562352	b'wish :d :

/tmp/ipython-input-57-289099189.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))


THE TWEET IS: Who's still awake ?:)))
THE PROCESSED TWEET IS: ["who'", 'still', 'awak', ':)']
1	0.49121759	b"who' still awak :)"
THE TWEET IS: @adoringpreston someone unfaved :((
THE PROCESSED TWEET IS: ['someon', 'unfav', ':(']
0	0.50000001	b'someon unfav :('
THE TWEET IS: @Quality_CE don't gas me :((
THE PROCESSED TWEET IS: ['ga', ':(']
0	0.50000001	b'ga :('
THE TWEET IS: as if not an apology isn't available in australia :(( @BeaMiller
THE PROCESSED TWEET IS: ['apolog', 'avail', 'australia', ':(']
0	0.50010730	b'apolog avail australia :('
THE TWEET IS: 😂😂 what a night :(((((( so devo xxx https://t.co/9ixTnyBXLb
THE PROCESSED TWEET IS: ['😂', '😂', 'night', ':(', 'devo', 'xxx']
0	0.50252757	b'  night :( devo xxx'
THE TWEET IS: @kingsxcreed daaaaaamn :((
THE PROCESSED TWEET IS: ['daaamn', ':(']
0	0.50000001	b'daaamn :('
THE TWEET IS: Dadas uh iphone na, :(:( [pic] — https://t.co/Jr4U98A8ja
THE PROCESSED TWEET IS: ['dada', 'uh', 'iphon', 'na', ':(', ':(', 'pic', '—']
0	0.50016314	b'dada u

/tmp/ipython-input-57-289099189.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))


THE PROCESSED TWEET IS: ['hit', 'thumb', ':(']
0	0.50110710	b'hit thumb :('
THE TWEET IS: Last day on the beach :-(:-(:-(.... http://t.co/oqo9F7FQs9
THE PROCESSED TWEET IS: ['last', 'day', 'beach', ':-(', ':-(', ':-(', '...']
0	0.50635527	b'last day beach :-( :-( :-( ...'
THE TWEET IS: @ranaPTX_ BUT WHO WILL LEAD US WHEN YOU'RE GONE :((
THE PROCESSED TWEET IS: ['lead', 'us', 'gone', ':(']
0	0.50561261	b'lead us gone :('
THE TWEET IS: @dongvvoo1122 😀🔫 r u sure u want that b r u h we barely survive w them in tank tops :((
THE PROCESSED TWEET IS: ['😀', '🔫', 'r', 'u', 'sure', 'u', 'want', 'b', 'r', 'u', 'h', 'bare', 'surviv', 'w', 'tank', 'top', ':(']
0	0.50486844	b'  r u sure u want b r u h bare surviv w tank top :('
THE TWEET IS: so thirsty :((
THE PROCESSED TWEET IS: ['thirsti', ':(']
0	0.50000001	b'thirsti :('
THE TWEET IS: where's all the jaebum baby pictures :((
THE PROCESSED TWEET IS: ["where'", 'jaebum', 'babi', 'pictur', ':(']
0	0.50012445	b"where' jaebum babi pictur :("


In [58]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else:
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[0.48723311]
Negative sentiment
